 <div class="alert alert-block alert-info"><font size="5"><center><b><u>Projecte Data Science </u></b></center></font>   
 <div class="alert alert-block alert-info"><font size="5"><center><b><u>CryptoPunks</u></b></center></font>   



\begin{align*}Anton\:Sobrino\end{align*}

\begin{align*}Juliol\hspace{2mm}\hspace{2mm}2021\end{align*}</div>

In [59]:
#Importació llibreries bàsiques 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import random
import re

#Importació Llibrereis per webscrapping
from selenium import webdriver

#Importacion 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time

#SQL
import mysql.connector
from sqlalchemy import create_engine

#labelencoder
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

driver = webdriver.Chrome(executable_path=r'C:\Users\Anton SL\bin_chrome\Chromedriver.exe')
#engine = create_engine('mysql+mysqlconnector://root:Zxcvbn123+-@' + 127001 +'/cryptopunks', echo=False)
time.sleep(random.uniform(2.0,2.0))


K = 1000
M = 1000000

# <span style=color:darkblue>A. Obtencio dades des de la web de Crypto Punks</span>

### <span style=color:darkred>1.tbl types of Crypto Punks</span>

In [60]:
url1 = 'https://www.larvalabs.com/cryptopunks/attributes'
driver.get(url1)


In [61]:
rows = len(driver.find_elements_by_xpath('/html/body/div[2]/div/div[3]/div/div[1]/table/tbody/tr'))
col = len(driver.find_elements_by_xpath('/html/body/div[2]/div/div[3]/div/div[1]/table/thead/tr[2]/th'))


print('col:', str(col), 'i rows:', str(rows) )

col: 6 i rows: 5


In [62]:
col_nom = []
for n in range(1,rows + 1):
    if n -1  < col :
        n == n-1
        col_nom.append(driver.find_element_by_xpath('/html/body/div[2]/div/div[3]/div/div[1]/table/thead/tr[2]/th[' + str(n) + ']').text)

col_nom

['Attribute', '#', 'Avail', 'Avg Sale', 'Cheapest']

In [63]:
MyListaAnidada = []
for n in range (1,rows + 1 ):
    mylista=[]
    for b in range (1, col):
        dato= driver.find_element_by_xpath('/html/body/div[2]/div/div[3]/div/div[1]/table/tbody/tr[' + str(n) + ']/td[' +str(b) +']').text
        mylista.append(dato)
        if len(mylista)== len(col_nom):
            MyListaAnidada.append(mylista)

df_types = pd.DataFrame(MyListaAnidada, columns = col_nom)
df_types.columns = ['Attribute', 'NumPunks', 'Avail', 'Avg_Sale_ETH', 'Cheapest_ETH']
#df_types['Avg_Sale_ETH'] = df_types['Avg_Sale_ETH'].str.extract(r'(0-9.\-)')
df_types['Avg_Sale_ETH']=df_types['Avg_Sale_ETH'].str.rstrip('Ξ') 
df_types['Avg_Sale_ETH']=df_types['Avg_Sale_ETH'].str.rstrip('K')
df_types['Cheapest_ETH']=df_types['Cheapest_ETH'].str.rstrip('Ξ') 
df_types['Cheapest_ETH']=df_types['Cheapest_ETH'].str.rstrip('K')
df_types['NumPunks']=pd.to_numeric(df_types['NumPunks'])
df_types['Avail']=pd.to_numeric(df_types['Avail'])
df_types['Avg_Sale_ETH']=pd.to_numeric(df_types['Avg_Sale_ETH']) 
df_types['Cheapest_ETH']=pd.to_numeric(df_types['Cheapest_ETH'])
df_types = df_types.fillna(0)
df_types.to_csv('types.csv')
df_types

,Attribute,NumPunks,Avail,Avg_Sale_ETH,Cheapest_ETH
0,Alien,9,0,0.00,0.0
1,Ape,24,4,0.00,1.6
2,Zombie,88,10,420.86,420.0
3,Female,3840,667,29.85,16.9
4,Male,6039,984,26.89,18.0


In [64]:
df_types.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Attribute     5 non-null      object 
 1   NumPunks      5 non-null      int64  
 2   Avail         5 non-null      int64  
 3   Avg_Sale_ETH  5 non-null      float64
 4   Cheapest_ETH  5 non-null      float64
dtypes: float64(2), int64(2), object(1)
memory usage: 328.0+ bytes


### <span style=color:darkred>2. tbl Top Sales</span>

In [69]:
url1 = 'https://www.larvalabs.com/cryptopunks/topsales?sortByUSD=false'
driver.get(url1)

In [70]:
MyListaAnidada=[]

for n in range(1, 100):
    try:
        CPunks = driver.find_element_by_xpath('/html/body/div[2]/div/div[2]/div['+ str(n)+ ']').text
        mylista = CPunks.split()
        MyListaAnidada.append(mylista[0:3])
    except:
        pass

df_topsales= pd.DataFrame(MyListaAnidada,columns =['Ranking','Num_CriptoPunk','Valor_ETH'])
df_topsales['Num_CriptoPunk'] = df_topsales['Num_CriptoPunk'].str.lstrip('#').astype('int64')
df_topsales['Valor_ETH']=df_topsales['Valor_ETH'].str.rstrip('Ξ') 
df_topsales['Valor_ETH']=df_topsales['Valor_ETH'].str.rstrip('K') 
df_topsales['Valor_ETH']=pd.to_numeric(df_topsales['Valor_ETH'])
df_topsales['Ranking']=pd.to_numeric(df_topsales['Ranking'])
df_topsales.to_csv('topsales.csv')
df_topsales.head(5)

,Ranking,Num_CriptoPunk,Valor_ETH
0,1,3100,4.2
1,2,7804,4.2
2,3,6965,800.0
3,4,2140,750.0
4,5,3011,667.0


In [71]:
df_topsales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Ranking         60 non-null     int64  
 1   Num_CriptoPunk  60 non-null     int64  
 2   Valor_ETH       60 non-null     float64
dtypes: float64(1), int64(2)
memory usage: 1.5 KB


### <span style=color:darkred>3. tbl CriptoPunks Transacciones</span>

In [72]:
url1 = 'https://www.larvalabs.com/cryptopunks/details/00001'
driver.get(url1)

In [73]:
col = len(driver.find_elements_by_xpath('//*[@id="punkHistory"]/div/table/tbody/tr[1]/th'))
rows = len(driver.find_elements_by_xpath('//*[@id="punkHistory"]/div/table/tbody/tr'))


print('col:', str(col), 'i rows:', str(rows) )

col: 5 i rows: 54


In [74]:
col_nom = ['Num_CP']

for n in range(1,col + 1):
    if n   < col + 1:
        n == n -1
        col_nom.append(driver.find_element_by_xpath('//*[@id="punkHistory"]/div/table/tbody/tr[1]/th[' + str(n) + ']').text)

col_nom

#//*[@id="punkHistory"]/div/table/tbody/tr[1]/th[1]

['Num_CP', 'Type', 'From', 'To', 'Amount', 'Txn']

### <span style=color:darkred>4. tbl Skin</span>

In [75]:
url1 = 'https://defypunk.com/punks/'
driver.get(url1)

In [76]:
skin=(driver.find_element_by_xpath('//*[@id="root"]/div/main/div/div/div[2]/div[2]/button')).text

In [77]:
mylista = []
MyListaAnidada=[]
df_skin = pd.DataFrame(columns=['skin','NumP', 'Punks'])

for i in range(1,10):
    try:
        skin=(driver.find_element_by_xpath('//*[@id="root"]/div/main/div/div/div[2]/div['+ str(i) +']/button')).text
        mylista.append(skin.split())
    except:
        pass
                       
df_skin = pd.DataFrame(mylista,columns=['skin','NumP', 'Punks'])
df_skin = df_skin.drop(['Punks'],axis=1)
df_skin['NumP']=df_skin['NumP'].str.replace('(','')
df_skin['NumP']=df_skin['NumP'].str.replace(',','')
df_skin['NumP']=pd.to_numeric(df_skin['NumP'])
df_skin.to_csv('skin.csv')
df_skin

,skin,NumP
0,Alien,9
1,Ape,24
2,Zombie,88
3,Albino,1018
4,Dark,2824
5,Light,3006
6,Mid,3031


In [78]:
df_skin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   skin    7 non-null      object
 1   NumP    7 non-null      int64 
dtypes: int64(1), object(1)
memory usage: 240.0+ bytes


### <span style=color:darkred>5. tbl Attibutes</span>

In [65]:
rows = len(driver.find_elements_by_xpath('/html/body/div[2]/div/div[3]/div/div[2]/table/tbody/tr'))
col = len(driver.find_elements_by_xpath('/html/body/div[2]/div/div[3]/div/div[2]/table/thead/tr[2]/th'))


print('col:', str(col), 'i rows:', str(rows) )

col: 6 i rows: 87


In [66]:
col_nom = []
for n in range(1,col + 1):
    if n   < col :
        n == n -1
        col_nom.append(driver.find_element_by_xpath('/html/body/div[2]/div/div[3]/div/div[2]/table/thead/tr[2]/th[' + str(n) + ']').text)

col_nom

['Attribute', '#', 'Avail', 'Avg Sale', 'Cheapest']

In [67]:
MyListaAnidada = []
for n in range (1,rows + 1 ):
    mylista=[]
    for b in range (1, col):
        dato= driver.find_element_by_xpath('/html/body/div[2]/div/div[3]/div/div[2]/table/tbody/tr[' + str(n) + ']/td[' +str(b) +']').text
        mylista.append(dato)
        if len(mylista)== len(col_nom):
            MyListaAnidada.append(mylista)
            
df_Attibutes = pd.DataFrame(MyListaAnidada, columns = col_nom)
df_Attibutes.columns = ['Attribute', 'NumPunks', 'Avail', 'Avg_Sale_ETH', 'Cheapest_ETH']
df_Attibutes['Avg_Sale_ETH']=df_Attibutes['Avg_Sale_ETH'].str.rstrip('Ξ') 
df_Attibutes['Avg_Sale_ETH']=df_Attibutes['Avg_Sale_ETH'].str.rstrip('K')
df_Attibutes['Cheapest_ETH']=df_Attibutes['Cheapest_ETH'].str.rstrip('Ξ') 
df_Attibutes['Cheapest_ETH']=df_Attibutes['Cheapest_ETH'].str.rstrip('K')
df_Attibutes['Avg_Sale_ETH']=pd.to_numeric(df_Attibutes['Avg_Sale_ETH'])
df_Attibutes['Cheapest_ETH']=pd.to_numeric(df_Attibutes['Cheapest_ETH'])
df_Attibutes['NumPunks']=pd.to_numeric(df_Attibutes['NumPunks'])
df_Attibutes['Avail']=pd.to_numeric(df_Attibutes['Avail'])
df_Attibutes = df_Attibutes.fillna(0)
df_Attibutes.to_csv('Attibutes.csv')
df_Attibutes

,Attribute,NumPunks,Avail,Avg_Sale_ETH,Cheapest_ETH
0,Beanie,44,11,152.00,199.99
1,Choker,48,7,67.33,75.00
2,Pilot Helmet,54,11,121.20,130.00
3,Tiara,55,15,67.00,73.88
4,Orange Side,68,15,96.03,65.56
...,...,...,...,...,...
82,Mole,644,111,23.75,16.90
83,Purple Lipstick,655,108,30.67,19.00
84,Hot Lipstick,696,124,30.10,19.00
85,Cigarette,961,168,28.43,18.80


In [68]:
df_Attibutes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87 entries, 0 to 86
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Attribute     87 non-null     object 
 1   NumPunks      87 non-null     int64  
 2   Avail         87 non-null     int64  
 3   Avg_Sale_ETH  87 non-null     float64
 4   Cheapest_ETH  87 non-null     float64
dtypes: float64(2), int64(2), object(1)
memory usage: 3.5+ KB


### <span style=color:darkred>6. tbl accesories</span>

In [175]:
punks_df= pd.read_csv('Punks.csv')

In [176]:
punks_df.head(5)

,Unnamed: 0,id,type,count,accesories
0,0,0,Female,3,Green Eye Shadow / Earring / Blonde Bob
1,1,1,Male,2,Smile / Mohawk
2,2,2,Female,1,Wild Hair
3,3,3,Male,3,Wild Hair / Nerd Glasses / Pipe
4,4,4,Male,4,Big Shades / Wild Hair / Earring / Goat


In [177]:
punks_df= punks_df.drop(['Unnamed: 0'], axis=1)

In [178]:
punks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          10000 non-null  int64 
 1   type        10000 non-null  object
 2   count       10000 non-null  int64 
 3   accesories  10000 non-null  object
dtypes: int64(2), object(2)
memory usage: 312.6+ KB


In [179]:
df = pd.DataFrame(punks_df.accesories)

In [180]:
df1 = df['accesories'].str.split('/', expand=True).stack()

In [181]:
df1

0     0     Green Eye Shadow 
      1              Earring 
      2            Blonde Bob
1     0                Smile 
      1                Mohawk
                  ...        
9998  0      Wild White Hair 
      1       Black Lipstick 
      2      Clown Eyes Green
9999  0               Mohawk 
      1          Nerd Glasses
Length: 27547, dtype: object

In [182]:
df1 = df1.reset_index(level=1,drop=True).rename('accesories1')

In [183]:
df1

0        Green Eye Shadow 
0                 Earring 
0               Blonde Bob
1                   Smile 
1                   Mohawk
               ...        
9998      Wild White Hair 
9998       Black Lipstick 
9998      Clown Eyes Green
9999               Mohawk 
9999          Nerd Glasses
Name: accesories1, Length: 27547, dtype: object

In [184]:
punks_df=punks_df.drop('accesories', axis=1)


In [185]:
punks_df = punks_df.join(df1)

In [186]:
punks_df

,id,type,count,accesories1
0,0,Female,3,Green Eye Shadow
0,0,Female,3,Earring
0,0,Female,3,Blonde Bob
1,1,Male,2,Smile
1,1,Male,2,Mohawk
...,...,...,...,...
9998,9998,Female,3,Wild White Hair
9998,9998,Female,3,Black Lipstick
9998,9998,Female,3,Clown Eyes Green
9999,9999,Female,2,Mohawk


In [187]:
punks_df['count'] = 1

In [188]:
#df1 = df1.reset_index(level=1,drop=True).rename('accesories1')
punks_df = punks_df.reset_index(drop=True)

In [189]:
#\w+\D
punks_df["type"] = punks_df["type"].str.extract(r'(\w+)')
punks_df["accesories1"] = punks_df["accesories1"].str.extract(r'(\w+\D+)')

In [190]:
punks_df

,id,type,count,accesories1
0,0,Female,1,Green Eye Shadow
1,0,Female,1,Earring
2,0,Female,1,Blonde Bob
3,1,Male,1,Smile
4,1,Male,1,Mohawk
...,...,...,...,...
27542,9998,Female,1,Wild White Hair
27543,9998,Female,1,Black Lipstick
27544,9998,Female,1,Clown Eyes Green
27545,9999,Female,1,Mohawk


In [191]:
punks_df.to_csv('punks_accesories.csv')

### <span style=color:darkred>6. tbl Master</span>

In [95]:
master_df = pd.DataFrame(range(0,10000), columns=['NumCriptoPunk'])
master_df.to_csv('master.csv')
master_df

,NumCriptoPunk
0,0
1,1
2,2
3,3
4,4
...,...
9995,9995
9996,9996
9997,9997
9998,9998


In [96]:
master_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 1 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   NumCriptoPunk  10000 non-null  int64
dtypes: int64(1)
memory usage: 78.2 KB


# <span style=color:darkblue>B. Creació BBDD MySql</span>

### <span style=color:darkred>Scripts MySQL </span>

### <span style=color:darkred>Scripts MySQL </span>

In [97]:
mydb = mysql.connector.connect(
  host="127.0.0.1",
  user="root",
  password="Zxcvbn123+-",
  database='cryptopunks'
)

print(mydb)

In [98]:
mycursor = mydb.cursor()

In [99]:
#df_topsales.to_sql(name='topsales', con=mydb, if_exists = 'append', index=False)